# This is a consolidated version of all the data


In [1]:
import base64
import datetime
import pandas as pd
import json
from google.cloud import bigquery

client = bigquery.Client(location="US")
print("Client creating using default project: {}".format(client.project))

Client creating using default project: research-311404


In [ ]:
# Construting Dependent variables: They should be one years looking forward: t+1
# Note that the 'fyear' here is just 
# 1. PatMV

t = 0
start_year = t + 1983
fyear = t + 1982
sql = f"""
    CREATE TABLE research-311404.AI2Patent_Tree_Consolidated_1year_forward.PatMV
    AS SELECT inventor_id, {fyear} AS fyear,  AVG(xi_real) AS xi_real FROM
    ((SELECT patent_id,inventor_id,year FROM  `research-311404.AI2Patent_Tree_Consolidated.AI2P_inventor_patent_level_min_full`)t1
    LEFT JOIN
    (SELECT CAST(patent_num AS STRING) AS patid,xi_real FROM `research-311404.KPSS_2022.KPSS_2022`)t3
    ON t1.patent_id = t3.patid
    )WHERE year = {start_year} 
    group by inventor_id
    """

job = client.query(sql)  # API request.
job.result()  # Waits for the query to finish.


In [4]:
for t in range(1,36):
    start_year = t + 1983
    fyear = t + 1982
    sql = f"""
        Insert `research-311404.AI2Patent_Tree_Consolidated_1year_forward.PatMV` 
        SELECT inventor_id, {fyear} AS fyear,  AVG(xi_real) AS xi_real FROM
        ((SELECT patent_id,inventor_id,year FROM  `research-311404.AI2Patent_Tree_Consolidated.AI2P_inventor_patent_level_min_full`)t1
        LEFT JOIN
        (SELECT CAST(patent_num AS STRING) AS patid,xi_real FROM `research-311404.KPSS_2022.KPSS_2022`)t3
        ON t1.patent_id = t3.patid
        )WHERE year = {start_year}
        group by inventor_id
        """

    job = client.query(sql)  # API request.
    job.result()  # Waits for the query to finish.


PatTV： 每个专利的TV为该专利在授权后3年内的引用数目/该专利申请当年同一类专利在未来3年的引用量的平均<br>
专利类别用的是cpc 分类的group id  -- 会有专利没有类别记录<br>
logistics： <br>
1. 确定专利的第一个cpc类别作为它的类别 
2. 确定在1980-2021年内申请的专利以及他们的类别 
3. 确定在1980-2021年申请的专利授权后36个月内的引用关系
4. 确定每个类别在1980-2021年申请的专利中平均每个的3年引用数量
5. 计算1980-2021申请的专利中每个的 scaled_TV
6. 计算inventor当年PatTV总和

In [5]:
# 2. PatTV
# t gets up to 2015

t = 0
start_year = t + 1983
fyear = t + 1982
sql = f"""
    CREATE TABLE research-311404.AI2Patent_Tree_Consolidated_1year_forward.PatTV
    AS SELECT inventor_id, {fyear} AS fyear,  AVG(citation_count) AS citation_count, AVG(scaled_citation) AS scaled_citation FROM
        ((SELECT  patent_id,inventor_id,year FROM  `research-311404.AI2Patent_Tree_Consolidated.AI2P_inventor_patent_level_min_full`)t1
        LEFT JOIN
        (SELECT patent_id,citation_count,scaled_citation FROM `research-311404.PatTV_Construction.scaled_citation_unique`)t3
        ON t1.patent_id = t3.patent_id
        )WHERE year = {start_year} 
        group by inventor_id
    """

job = client.query(sql)  # API request.
job.result()  # Waits for the query to finish.


for t in range(1,34):
    start_year = t + 1983
    fyear = t + 1982
    sql = f"""
        INSERT `research-311404.AI2Patent_Tree_Consolidated_1year_forward.PatTV`
        SELECT inventor_id, {fyear} AS fyear,  AVG(citation_count) AS citation_count, AVG(scaled_citation) AS scaled_citation FROM
        ((SELECT  patent_id,inventor_id,year FROM  `research-311404.AI2Patent_Tree_Consolidated.AI2P_inventor_patent_level_min_full`)t1
        LEFT JOIN
        (SELECT patent_id,citation_count,scaled_citation FROM `research-311404.PatTV_Construction.scaled_citation_unique`)t3
        ON t1.patent_id = t3.patent_id
        )WHERE year = {start_year} 
        group by inventor_id
        """

    job = client.query(sql)  # API request.
    job.result()  # Waits for the query to finish.


In [7]:
# 3. PatNum
t = 0
start_year = t + 1983
fyear = t + 1982
sql = f"""
CREATE TABLE
  research-311404.AI2Patent_Tree_Consolidated_1year_forward.PatNum AS
    SELECT inventor_id, {fyear} AS fyear,  IFNULL(COUNT(patent_id),0) AS PatNum FROM `research-311404.AI2Patent_Tree_Consolidated.AI2P_inventor_patent_level_min_full`
        WHERE year = {start_year} 
        group by inventor_id
"""
job = client.query(sql)  # API request.
job.result()
    
for t in range(1,37):
    start_year = t + 1983
    fyear = t + 1982
    sql = f"""
     INSERT `research-311404.AI2Patent_Tree_Consolidated_1year_forward.PatNum`
        SELECT inventor_id, {fyear} AS fyear,  IFNULL(COUNT(patent_id),0) AS PatNum FROM `research-311404.AI2Patent_Tree_Consolidated.AI2P_inventor_patent_level_min_full`
            WHERE year = {start_year} 
            group by inventor_id
    """
    job = client.query(sql)  # API request.
    job.result()

In [9]:
## Exploration: three year aggregation
# Exploration patents should be good when t+1 >= 1983 AND t+3 <= 2019
# Therefore available t values are from 1982 to 2016
    
t = 0
start_year = t + 1983
fyear = t + 1982
sql = f"""
CREATE TABLE
  research-311404.AI2Patent_Tree_Consolidated_1year_forward.Exploration AS
        SELECT inventor_id, {fyear} AS fyear,  AVG(exploration) AS exploration FROM
        ((SELECT patent_id,inventor_id,year FROM  `research-311404.AI2Patent_Tree_Consolidated.AI2P_inventor_patent_level_min_full`)t1
        LEFT JOIN
        (SELECT patentid,exploration FROM `research-311404.database.Exploration_all`)t3
        ON t1.patent_id = t3.patentid )
         WHERE year = {start_year} 
            group by inventor_id
"""
job = client.query(sql)  # API request.
job.result()
    
for t in range(1,35):
    start_year = t + 1983
    fyear = t + 1982
    sql = f"""
     INSERT `research-311404.AI2Patent_Tree_Consolidated_1year_forward.Exploration`
        SELECT inventor_id, {fyear} AS fyear,  AVG(exploration) AS exploration FROM
        ((SELECT patent_id,inventor_id,year FROM  `research-311404.AI2Patent_Tree_Consolidated.AI2P_inventor_patent_level_min_full`)t1
        LEFT JOIN
        (SELECT patentid,exploration FROM `research-311404.database.Exploration_all`)t3
        ON t1.patent_id = t3.patentid )
        WHERE year = {start_year} 
        group by inventor_id
    """
    job = client.query(sql)  # API request.
    job.result()

In [10]:
## 5. CD Index
## Original CD Index data has information on 5,894,529 patents
## We build this based on mcd_5
## This information only goes from 1980 to 2013
## Therefore we can only limit t from 1982 to 2010

    
t = 0
start_year = t + 1983
fyear = t + 1982
sql = f"""
CREATE TABLE
  research-311404.AI2Patent_Tree_Consolidated_1year_forward.CDIndex AS
        SELECT inventor_id, {fyear} AS fyear,  AVG(mcd_5) AS mcd_5 FROM
        ((SELECT patent_id,inventor_id,year FROM  `research-311404.AI2Patent_Tree_Consolidated.AI2P_inventor_patent_level_min_full`)t1
        LEFT JOIN
        (SELECT CAST(patent_number AS STRING) AS patent_number,mcd_5 FROM `research-311404.CDIndex.cdindex_cross` )t3
        ON t1.patent_id = t3.patent_number
        )WHERE year = {start_year}
        group by inventor_id
"""
job = client.query(sql)  # API request.
job.result()
    
for t in range(1,29):
    start_year = t + 1983
    fyear = t + 1982
    sql = f"""
     INSERT `research-311404.AI2Patent_Tree_Consolidated_1year_forward.CDIndex`
        SELECT inventor_id, {fyear} AS fyear,  AVG(mcd_5) AS mcd_5 FROM
        ((SELECT patent_id,inventor_id,year FROM  `research-311404.AI2Patent_Tree_Consolidated.AI2P_inventor_patent_level_min_full`)t1
        LEFT JOIN
        (SELECT CAST(patent_number AS STRING) AS patent_number,mcd_5 FROM `research-311404.CDIndex.cdindex_cross` )t3
        ON t1.patent_id = t3.patent_number
        )WHERE year = {start_year}
        group by inventor_id
    """
    job = client.query(sql)  # API request.
    job.result()

In [11]:
## 6. RETech & Breadth
## 9,575,245 data available from RETECH original file
## data's time range is 1980-2017.
## Therefore we can only limit t from 1982 to 2014 
    
t = 0
start_year = t + 1983
fyear = t + 1982
sql = f"""
CREATE TABLE
  research-311404.AI2Patent_Tree_Consolidated_1year_forward.RETech AS
        SELECT inventor_id,{fyear} AS fyear, AVG(RETech) AS RETech,AVG(Breadth) AS Breadth  FROM
        ((SELECT patent_id,inventor_id,year FROM  `research-311404.AI2Patent_Tree_Consolidated.AI2P_inventor_patent_level_min_full`)t1
        LEFT JOIN
        (SELECT CAST(pnum AS STRING) AS patent_number,RETech,Breadth FROM `research-311404.TechScopeBreadth.raw` )t3
        ON t1.patent_id = t3.patent_number
        )
        WHERE RETech IS NOT NULL AND Breadth IS NOT NULL AND year = {start_year} 
        GROUP BY inventor_id
"""
job = client.query(sql)  # API request.
job.result()
    
for t in range(1,33):
    start_year = t + 1983
    fyear = t + 1982
    sql = f"""
     INSERT `research-311404.AI2Patent_Tree_Consolidated_1year_forward.RETech`
        SELECT inventor_id,{fyear} AS fyear, AVG(RETech) AS RETech,AVG(Breadth) AS Breadth  FROM
        ((SELECT patent_id,inventor_id,year FROM  `research-311404.AI2Patent_Tree_Consolidated.AI2P_inventor_patent_level_min_full`)t1
        LEFT JOIN
        (SELECT CAST(pnum AS STRING) AS patent_number,RETech,Breadth FROM `research-311404.TechScopeBreadth.raw` )t3
        ON t1.patent_id = t3.patent_number
        )
        WHERE RETech IS NOT NULL AND Breadth IS NOT NULL AND year = {start_year} 
        GROUP BY inventor_id
    """
    job = client.query(sql)  # API request.
    job.result()

In [12]:
## 7. Generality & Originality
## 6,330,939 rows on originality (backward citation)
## 5,123,492 rows on generality (forward citation)
## Start with originality
## Originality ranges from 1980 - 2021. Therefore allowing for at least 3 years (t + 1 to t + 3), t's range is from 1982 to 2018

t = 0
start_year = t + 1983
fyear = t + 1982
sql = f"""
CREATE TABLE
  research-311404.AI2Patent_Tree_Consolidated_1year_forward.originality AS
        AS SELECT inventor_id,{fyear} AS fyear, AVG(originality) AS originality FROM
        ((SELECT patent_id,inventor_id,year FROM  `research-311404.AI2Patent_Tree_Consolidated.AI2P_inventor_patent_level_min_full`)t1
        LEFT JOIN
        (SELECT patent_id,originality FROM `research-311404.Generality_Consolidated.originality` )t3
        ON t1.patent_id = t3.patent_id
        )
        WHERE originality IS NOT NULL AND year = {start_year}
        GROUP BY inventor_id
"""
job = client.query(sql)  # API request.
job.result()
    
for t in range(1,37):
    start_year = t + 1983
    fyear = t + 1982
    sql = f"""
     INSERT `research-311404.AI2Patent_Tree_Consolidated_1year_forward.originality`
        SELECT inventor_id,{fyear} AS fyear, AVG(originality) AS originality FROM
        ((SELECT patent_id,inventor_id,year FROM  `research-311404.AI2Patent_Tree_Consolidated.AI2P_inventor_patent_level_min_full`)t1
        LEFT JOIN
        (SELECT patent_id,originality FROM `research-311404.Generality_Consolidated.originality` )t3
        ON t1.patent_id = t3.patent_id
        )
        WHERE originality IS NOT NULL AND year = {start_year}
        GROUP BY inventor_id
    """
    job = client.query(sql)  # API request.
    job.result()

BadRequest: 400 Syntax error: Unexpected keyword AS at [4:9]

Location: US
Job ID: 061b9f35-d9e2-4fcc-9e08-ff4ee9cea617


In [ ]:
## Generality ranges from 1980 - 2021. Therefore allowing for at least 3 years (t + 1 to t + 3), t's range is from 1982 to 2015
t = 0
start_year = t + 1983
fyear = t + 1982
sql = f"""
CREATE TABLE
  research-311404.AI2Patent_Tree_Consolidated_1year_forward.generality 
        AS  SELECT inventor_id,{fyear} AS fyear, AVG(generality) AS generality FROM
        ((SELECT patent_id,inventor_id,year FROM  `research-311404.AI2Patent_Tree_Consolidated.AI2P_inventor_patent_level_min_full`)t1
        LEFT JOIN
        (SELECT patent_id,generality FROM `research-311404.Generality_Consolidated.generality` )t3
        ON t1.patent_id = t3.patent_id
        )
        WHERE generality IS NOT NULL AND year = {start_year}
        GROUP BY inventor_id
"""
job = client.query(sql)  # API request.
job.result()
    
for t in range(1,34):
    start_year = t + 1983
    fyear = t + 1982
    sql = f"""
     INSERT `research-311404.AI2Patent_Tree_Consolidated_1year_forward.generality`
        SELECT inventor_id,{fyear} AS fyear, AVG(generality) AS generality FROM
        ((SELECT patent_id,inventor_id,year FROM  `research-311404.AI2Patent_Tree_Consolidated.AI2P_inventor_patent_level_min_full`)t1
        LEFT JOIN
        (SELECT patent_id,generality FROM `research-311404.Generality_Consolidated.generality` )t3
        ON t1.patent_id = t3.patent_id
        )
        WHERE generality IS NOT NULL AND year = {start_year}
        GROUP BY inventor_id
    """
    job = client.query(sql)  # API request.
    job.result()

In [16]:
for t in range(1,34):
    start_year = t + 1983
    fyear = t + 1982
    sql = f"""
     INSERT `research-311404.AI2Patent_Tree_Consolidated_1year_forward.generality`
        SELECT inventor_id,{fyear} AS fyear, AVG(generality) AS generality FROM
        ((SELECT patent_id,inventor_id,year FROM  `research-311404.AI2Patent_Tree_Consolidated.AI2P_inventor_patent_level_min_full`)t1
        LEFT JOIN
        (SELECT patent_id,generality FROM `research-311404.Generality_Consolidated.generality` )t3
        ON t1.patent_id = t3.patent_id
        )
        WHERE generality IS NOT NULL AND year = {start_year}
        GROUP BY inventor_id
    """
    job = client.query(sql)  # API request.
    job.result()